# 🗺️ Interactive Mapping with Folium - Hospital Access in Peru

## 📋 Overview

This notebook implements interactive mapping for hospital accessibility analysis in Peru using Folium. The analysis focuses on two main components:

### 🎯 Tasks:
1. **National Choropleth (District Level)**: Folium choropleth showing number of hospitals per district with marker clusters
2. **Proximity Visualization**: Lima & Loreto analysis with 10km radius circles for population centers

### 🔍 Key Features:
- Interactive choropleth maps at district level
- Hospital marker clustering for better visualization
- Proximity analysis with red/green circles (10km radius)
- Comprehensive popups and tooltips
- Comparative analysis: Urban (Lima) vs Amazon (Loreto)

---

## 1. 📚 Setup and Data Loading

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('src')  # Add src directory to path

import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import plugins
import warnings
warnings.filterwarnings('ignore')

# Import our custom utility functions
from utils import (
    load_and_clean_data,
    filter_operational_hospitals,
    spatial_join_hospitals_districts,
    perform_proximity_analysis,
    create_national_choropleth_folium,
    create_proximity_folium_map
)

print("✅ Libraries and utilities imported successfully")
print("🔄 Loading and preprocessing data...")

In [ ]:
# Load and preprocess all datasets
hospitals_df, districts_gdf, pop_centers_gdf = load_and_clean_data()

# Filter operational hospitals
public_hospitals, hospitals_gdf = filter_operational_hospitals(hospitals_df)

# Perform spatial join to count hospitals per district
districts_with_counts = spatial_join_hospitals_districts(hospitals_gdf, districts_gdf)

print(f"\n📊 Data Summary:")
print(f"   • Total operational hospitals: {len(hospitals_gdf):,}")
print(f"   • Districts: {len(districts_gdf):,}")
print(f"   • Population centers: {len(pop_centers_gdf):,}")
print(f"   • Districts with hospitals: {len(districts_with_counts[districts_with_counts['hospital_count'] > 0]):,}")
print("\n✅ Data loading completed successfully!")

---
## 2. 🗺️ Task 1: National Choropleth (District Level)

### Objective: 
Build a comprehensive Folium choropleth map showing:
- Number of hospitals per district (color-coded)
- Hospital markers with clustering for better performance
- Interactive tooltips and popups

In [ ]:
# Create the national choropleth map with hospital markers
print("🔄 Creating national choropleth map...")

national_map = create_national_choropleth_folium(districts_with_counts, hospitals_gdf)

# Add professional title and context
title_html = '''
<h3 style="position: fixed; 
           top: 10px; left: 50px; width: 450px; height: 80px; 
           background-color: white; border:2px solid grey; z-index:9999; 
           font-size:16px; text-align: center; padding: 10px">
<b>🏥 National Hospital Distribution - Peru 2024</b><br>
<small>District-level choropleth with clustered hospital markers</small><br>
<small style="color: #666;">📊 Data: MINSA | 🗺️ Interactive Folium Map</small>
</h3>
'''
national_map.get_root().html.add_child(folium.Element(title_html))

# Add summary statistics box
stats_html = f'''
<div style="position: fixed; 
            bottom: 10px; left: 10px; width: 300px; height: 120px; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:12px; padding: 10px">
<b>📈 Summary Statistics</b><br>
• Total Hospitals: {len(hospitals_gdf):,}<br>
• Districts: {len(districts_gdf):,}<br>
• Districts with hospitals: {len(districts_with_counts[districts_with_counts['hospital_count'] > 0]):,}<br>
• Max hospitals per district: {districts_with_counts['hospital_count'].max()}<br>
• Avg hospitals per district: {districts_with_counts['hospital_count'].mean():.1f}
</div>
'''
national_map.get_root().html.add_child(folium.Element(stats_html))

print("✅ National choropleth map created successfully!")
print("\n🔍 Map Features:")
print("   • Color-coded districts by hospital count")
print("   • Clustered hospital markers for performance")
print("   • Interactive popups with hospital details")
print("   • Professional legend and statistics")

# Display the map
national_map

---
## 3. 📍 Task 2: Proximity Visualization - Lima & Loreto

### Objective:
Reproduce proximity analysis comparing urban (Lima) vs Amazon (Loreto) accessibility:
- **Red circles (10 km)**: Population centers with LOWEST hospital density
- **Green circles (10 km)**: Population centers with HIGHEST hospital density
- Interactive popups with population center names and hospital counts

### 3.1 Lima Analysis (Urban Concentration)

In [ ]:
# Perform proximity analysis for Lima
print("🔄 Analyzing proximity for Lima (Urban concentration)...")

lima_isolated, lima_concentrated, lima_hospitals = perform_proximity_analysis(
    hospitals_gdf, pop_centers_gdf, 'Lima', buffer_km=10
)

if lima_isolated is not None and lima_concentrated is not None:
    # Create Lima proximity map
    lima_center = [-12.0464, -77.0428]  # Lima coordinates
    lima_map = create_proximity_folium_map(
        lima_hospitals, lima_isolated, lima_concentrated,
        'Lima', lima_center, zoom=9
    )
    
    # Add analysis context for Lima
    lima_analysis_html = f'''
    <div style="position: fixed; 
                bottom: 10px; left: 10px; width: 320px; height: 200px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:11px; padding: 8px">
    <b>🏙️ Lima Urban Analysis</b><br><br>
    <b>Most Isolated Center:</b><br>
    • Name: {lima_isolated.get('nome', 'Unknown')}<br>
    • Hospitals in 10km: {lima_isolated['hospitals_10km']}<br><br>
    <b>Most Concentrated Center:</b><br>
    • Name: {lima_concentrated.get('nome', 'Unknown')}<br>
    • Hospitals in 10km: {lima_concentrated['hospitals_10km']}<br><br>
    <b>Key Insights:</b><br>
    • Urban concentration enables better accessibility<br>
    • Clear disparity even within metro area
    </div>
    '''
    lima_map.get_root().html.add_child(folium.Element(lima_analysis_html))
    
    print("✅ Lima proximity analysis completed!")
    print(f"   • Most isolated center: {lima_isolated['hospitals_10km']} hospitals within 10km")
    print(f"   • Most concentrated center: {lima_concentrated['hospitals_10km']} hospitals within 10km")
    print(f"   • Total Lima hospitals: {len(lima_hospitals)}")
else:
    print("❌ Lima analysis data not available")
    lima_map = None

In [ ]:
# Display Lima proximity map
if lima_map is not None:
    print("🗺️ Lima Proximity Analysis Map:")
    lima_map
else:
    print("⚠️ Lima map not available")

### 3.2 Loreto Analysis (Amazon Challenges)

In [ ]:
# Perform proximity analysis for Loreto
print("🔄 Analyzing proximity for Loreto (Amazon challenges)...")

loreto_isolated, loreto_concentrated, loreto_hospitals = perform_proximity_analysis(
    hospitals_gdf, pop_centers_gdf, 'Loreto', buffer_km=10
)

if loreto_isolated is not None and loreto_concentrated is not None:
    # Create Loreto proximity map
    loreto_center = [-4.2312, -73.2516]  # Loreto coordinates
    loreto_map = create_proximity_folium_map(
        loreto_hospitals, loreto_isolated, loreto_concentrated,
        'Loreto', loreto_center, zoom=7
    )
    
    # Add analysis context for Loreto
    loreto_analysis_html = f'''
    <div style="position: fixed; 
                bottom: 10px; left: 10px; width: 320px; height: 220px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:11px; padding: 8px">
    <b>🌳 Loreto Amazon Analysis</b><br><br>
    <b>Most Isolated Center:</b><br>
    • Name: {loreto_isolated.get('nome', 'Unknown')}<br>
    • Hospitals in 10km: {loreto_isolated['hospitals_10km']}<br><br>
    <b>Most Concentrated Center:</b><br>
    • Name: {loreto_concentrated.get('nome', 'Unknown')}<br>
    • Hospitals in 10km: {loreto_concentrated['hospitals_10km']}<br><br>
    <b>Amazon Challenges:</b><br>
    • Geographic dispersion limits access<br>
    • River-based transportation dependency<br>
    • Need for specialized rural solutions
    </div>
    '''
    loreto_map.get_root().html.add_child(folium.Element(loreto_analysis_html))
    
    print("✅ Loreto proximity analysis completed!")
    print(f"   • Most isolated center: {loreto_isolated['hospitals_10km']} hospitals within 10km")
    print(f"   • Most concentrated center: {loreto_concentrated['hospitals_10km']} hospitals within 10km")
    print(f"   • Total Loreto hospitals: {len(loreto_hospitals)}")
else:
    print("❌ Loreto analysis data not available")
    loreto_map = None

In [ ]:
# Display Loreto proximity map
if loreto_map is not None:
    print("🗺️ Loreto Proximity Analysis Map:")
    loreto_map
else:
    print("⚠️ Loreto map not available")

---
## 4. 📊 Comparative Analysis: Lima vs Loreto

### Written Analysis

Based on the interactive proximity analysis with 10km buffers, we observe significant contrasts between urban and Amazon regions:

In [ ]:
# Generate comparative statistics
if lima_isolated is not None and loreto_isolated is not None:
    print("📈 COMPARATIVE ANALYSIS: LIMA vs LORETO")
    print("=" * 50)
    
    print("\n🏙️ LIMA (Urban Concentration):")
    print(f"   • Total hospitals: {len(lima_hospitals)}")
    print(f"   • Most isolated center: {lima_isolated['hospitals_10km']} hospitals in 10km")
    print(f"   • Most concentrated center: {lima_concentrated['hospitals_10km']} hospitals in 10km")
    lima_ratio = lima_concentrated['hospitals_10km'] / max(lima_isolated['hospitals_10km'], 1)
    print(f"   • Concentration ratio: {lima_ratio:.1f}:1")
    
    print("\n🌳 LORETO (Amazon Dispersion):")
    print(f"   • Total hospitals: {len(loreto_hospitals)}")
    print(f"   • Most isolated center: {loreto_isolated['hospitals_10km']} hospitals in 10km")
    print(f"   • Most concentrated center: {loreto_concentrated['hospitals_10km']} hospitals in 10km")
    loreto_ratio = loreto_concentrated['hospitals_10km'] / max(loreto_isolated['hospitals_10km'], 1)
    print(f"   • Concentration ratio: {loreto_ratio:.1f}:1")
    
    print("\n🔍 KEY INSIGHTS:")
    print(f"   • Hospital density difference: {len(lima_hospitals) - len(loreto_hospitals)} hospitals")
    print(f"   • Lima concentration advantage: {lima_ratio/loreto_ratio:.1f}x better")
    print("   • Geographic factors significantly impact accessibility")
    
else:
    print("⚠️ Comparative analysis not available - missing data for one or both regions")

### 🎯 Policy Implications

#### **Lima: Urban Concentration and Accessibility**
- **Strengths**: High hospital density enables better access within metro areas
- **Challenges**: Disparity exists even within urban zones
- **Recommendations**: 
  - Optimize hospital distribution in underserved urban peripheries
  - Enhance public transportation to hospital districts
  - Develop specialized urban health centers

#### **Loreto: Geographic Dispersion and Accessibility Challenges in the Amazon**
- **Challenges**: Vast geographic distances and limited infrastructure
- **Geographic barriers**: Rivers, forests, and remote communities
- **Recommendations**:
  - **Telemedicine programs** for remote consultations
  - **Mobile river units** for waterway-accessible communities
  - **Strategic health posts** in key population centers
  - **Air medical transport** for emergency cases

#### **Methodology Validation: 10km Buffer Analysis**
The 10km buffer analysis effectively reveals:
- **Urban vs Rural disparities** in hospital accessibility
- **Geographic constraints** that traditional distance metrics cannot capture
- **Population center vulnerability** in terms of health service access
- **Need for context-specific solutions** based on regional characteristics

---

### 📝 Conclusions

1. **Urban-Rural Divide**: Clear accessibility gap between metropolitan Lima and Amazon Loreto
2. **Geographic Determinism**: Physical geography significantly shapes health access patterns
3. **Policy Differentiation**: One-size-fits-all approaches are insufficient
4. **Technology Integration**: Modern solutions (telemedicine, mobile units) essential for Amazon regions
5. **Strategic Planning**: Hospital placement must consider population distribution and geographic constraints

---
## 5. 🔧 Technical Summary

### Map Features Implemented:
✅ **National Choropleth (Task 1)**:
- District-level color coding by hospital count
- MarkerCluster for hospital points
- Interactive popups with hospital details
- Professional styling and statistics

✅ **Proximity Visualization (Task 2)**:
- Red circles for isolated population centers
- Green circles for concentrated population centers
- 10km radius buffers
- Detailed popups with population center names and hospital counts
- Comparative analysis for Lima vs Loreto

### Technical Stack:
- **Folium**: Interactive web mapping
- **GeoPandas**: Spatial data processing
- **Pandas**: Data manipulation
- **Custom utilities**: Reusable analysis functions

### Data Quality:
- **MINSA IPRESS**: Hospital registry (⭐⭐⭐⭐⭐)
- **INEI Population Centers**: CCPP_IGN100K (⭐⭐⭐⭐⭐)
- **IGN Districts**: Administrative boundaries (⭐⭐⭐⭐⭐)

---

## 🎉 Interactive Mapping Complete!

This notebook successfully implements comprehensive interactive mapping for hospital accessibility analysis in Peru, providing valuable insights for health policy and planning decisions.